In [ ]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

# Find the repository root by searching upward for a 'pyFMM' directory
start = Path.cwd().resolve()
repo_root = None
for p in [start] + list(start.parents):
    if (p / 'pyFMM').is_dir():
        repo_root = str(p)
        break
if repo_root is None:
    raise RuntimeError("Could not find 'pyFMM' in any parent directory of cwd")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
from pyFMM import *
import numpy as np

In [ ]:
np.random.seed(42)
N = 10000                              # number of points
q = np.random.choice([-1, 1], size=N)  # source strengths with magnitude 1 and random sign
#q = np.ones(N)                        # source strengths all positive 1
source_area_size =  1.0
#---------- moment order ---------
p = 4
#---------------------------------

In [ ]:
#--------- generate a number of random points in side box ----------
# NOTE - with this we get slight asymmetries compared to spherical distribution
#       this meains higher order moments are more imporant
LLC = np.array([ -1.0, -1.0, -1.0 ]) * source_area_size   # Lower Left Corner
URC = np.array([  1.0,  1.0,  1.0 ]) * source_area_size   # Upper Right Corner
X = np.random.rand(N, 3)             # uniform in [0,1)
X = LLC + (URC - LLC) * X            # uniform in [LLC, URC]
#------------------------------------------------------------------
#--------- convert to polar coordinates ----------------
x_polar = cart_to_sphe(X)
#------------------------------------------------------

In [ ]:
#--------- generate target points ----------------
# NOTE - for this we also generate target points in a box
Ntgt = N
dist = 10 * source_area_size
center = np.array([1.0, 1.0, 1.0]) * dist
x_tgt = center + (np.random.rand(Ntgt, 3) - 0.5) * 2.0 * source_area_size
x_tgt_polar = cart_to_sphe(x_tgt)
#--------------------------------------------------

In [ ]:
# child centres at the 8 octants
child_centres_rel = np.array([[-0.5, -0.5, -0.5],
                 [ 0.5, -0.5, -0.5],
                 [-0.5,  0.5, -0.5],
                 [ 0.5,  0.5, -0.5],
                 [-0.5, -0.5,  0.5],
                 [ 0.5, -0.5,  0.5],
                 [-0.5,  0.5,  0.5],
                 [ 0.5,  0.5,  0.5]]) * source_area_size 


child_centres = child_centres_rel+ center

# assign sources to children by sign of coordinates
child_masks = []
for cx in child_centres:
    x_mask = np.logical_and(x_tgt[:,0] >= cx[0] - source_area_size *0.5, x_tgt[:,0] <= cx[0] + source_area_size * 0.5)   
    y_mask = np.logical_and(x_tgt[:,1] >= cx[1] - source_area_size *0.5, x_tgt[:,1] <= cx[1] + source_area_size * 0.5)
    z_mask = np.logical_and(x_tgt[:,2] >= cx[2] - source_area_size *0.5, x_tgt[:,2] <= cx[2] + source_area_size * 0.5)
    mask = np.logical_and(np.logical_and(x_mask, y_mask), z_mask)

    child_masks.append(mask)

In [ ]:
# build one array per child
child_points = [x_tgt[mask] for mask in child_masks]

# pick a distinct colour per child
child_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
                'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray']

# now call plot_points_3d with *all* children plus the targets
fig, ax = plot_points_3d(
    X, *child_points,   # unpack each child’s points as its own arg
    labels=['Source'] + [f'Taget Child {i+1}' for i in range(len(child_points))],
    colors=['black'] + child_colors ,   # black or red for targets
    sizes=[2.0] + [0.5]*len(child_points),
    title='3D Source and Target Distribution (Octants)',
)


In [ ]:
#---------- compute moments and potentials ----------
M_std = P2M_sphe(x_polar, q, p=p)
P_mom = P_sphe(M_std, x_tgt_polar)
P_dir_vec =  P_direct_cart(X, q, x_tgt, eps=1e-10)
#------------------------------------------------------

In [ ]:

x0 = np.array([0.0, 0.0, 0.0])
x1 = center

L = M2L_sphe(M_std, x0, x1)
x_tgt_rel = x_tgt - x1
x_tgt_rel_polar = cart_to_sphe(x_tgt_rel)
P_local = P_L_sphe(L, x_tgt_rel_polar)

In [ ]:
#------------ perform L2L translations to each child centre -------------
L_all = []
for cx in child_centres:
    x0 = center
    x1 = cx
    L_child = L2L_sphe(L, x0, x1)
    L_all.append(L_child)
L_all = np.array(L_all)
#-----------------------------------------------------------------------


P_L2L = np.zeros_like(P_local)
#---------- evaluate potential based on child local expansions -------------
for i, cx in enumerate(child_centres):
    x_child = x_tgt[child_masks[i]]
    x_child_rel = x_child - cx
    x_child_rel_polar = cart_to_sphe(x_child_rel)
    P_child = P_L_sphe(L_all[i], x_child_rel_polar)
    #--------- cast back to global potential array -------------
    P_L2L[child_masks[i]] = P_child
    #-----------------------------------------------------------
#-----------------------------------------------------------------------

In [ ]:
#--------- comput relative error between direct summation and moment evaluation ----------
relative_errors = np.abs(P_dir_vec - P_local) / np.abs(P_dir_vec)
relative_errors_all = np.abs(P_dir_vec - P_L2L) / np.abs(P_dir_vec)

#------------------------------------------------------
fig, ax = plot_ylog(
    relative_errors, relative_errors_all,
    title='Relative Error per Target',
    xlabel='Target Index',
    ylabel='Relative Error (log scale)',
    sizes=[2,2],
    labels=["M_L", "M_L2L"],
    legend=True
)
plt.show()

In [ ]:
np.random.seed(42)
rel_point_L = np.random.choice([-1, 1], size=3) * source_area_size * 0.1
rel_point_L_sphe = cart_to_sphe(np.array([rel_point_L]))
point_L = center + rel_point_L

child_index = np.argmin(np.linalg.norm(child_centres_rel - rel_point_L, axis=1))
#child_index = np.argmin(np.linalg.norm(child_centres - point_L, axis=1))

rel_point_child = point_L - child_centres[child_index]
rel_point_child_sphe = cart_to_sphe(np.array([rel_point_child]))

P_point_dir = P_direct_cart(X, q, point_L.reshape(1,3), eps=1e-10)[0]
P_point_L = P_L_sphe(L, rel_point_L_sphe)[0]
P_point_child = P_L_sphe(L_all[child_index], rel_point_child_sphe)[0]





print("Potential at point via direct summation:         ", P_point_dir)
print("Potential at point via parent local expansion:   ", P_point_L)
print("Potential at point via child local expansion:    ", P_point_child)
print("")
print("diff parent-local - direct:                      ", np.abs(P_point_L - P_point_dir))
print("diff child-local - direct:                       ", np.abs(P_point_child - P_point_dir))

In [ ]:
#--------------- create list of p order and separation distances --------------
all_p = np.arange(1, 10)            
sep_dist = np.arange(2,16,2) * source_area_size
#-------------------------------------------------------------------------------
#---------- allocate arrays for results ----------
P_loc = np.zeros((len(all_p), len(sep_dist)))
P_dir = np.zeros((len(all_p), len(sep_dist)))
#------------------------------------------------
#----------------- center point for origin of moments ----------------
x0 = np.array([0.0, 0.0, 0.0])
#--------------------------------------------------------------------
for ip, p in enumerate(all_p):
    #------------ compute moments at this p --------------
    M_x0 = P2M_sphe(x_polar, q, p=p)
    #-----------------------------------------------------
    for jd, dist in enumerate(sep_dist):
        #--------- define center for parent local expansion ----------
        center = np.array([1.0, 1.0, 1.0]) * dist
        #-------------------------------------------------------------
        #----------- calculate local expansion at parent centre -------------
        L_parent = M2L_sphe(M_x0, x0, center)
        #--------------------------------------------------------------------
        #------------ define child centre and compute L2L --------------
        x1 = child_centres_rel[child_index] + center
        L_child = L2L_sphe(L_parent, center, x1)
        #---------------------------------------------------------------
        #-------- define evaluation point ----------
        eval_point = center + rel_point_L
        #-----------------------------------------
        #------------ define evaluation point relative to child centre ----------
        eval_point_rel = eval_point - x1
        eval_point_rel_sphe = cart_to_sphe(np.array([eval_point_rel]))[0]
        #------------------------------------------------------------------------
        #-------------- calculate direct and local potentials ----------------
        P_dir[ip, jd] = P_direct_cart(X, q, np.array([eval_point]), eps=1e-10)[0]
        P_loc[ip, jd] = P_L_sphe(L_child, eval_point_rel_sphe)[0]
        #--------------------------------------------------------------------
#----------- compute errors ----------------
abs_err = np.abs(P_loc - P_dir)
with np.errstate(divide='ignore', invalid='ignore'):
    rel_err = abs_err / np.maximum(np.abs(P_dir), 1e-300)  # robust denominator
#--------------------------------------------


In [ ]:
#---------- first plot to verify parity --------------
plt.figure(figsize=(8,6))
plt.scatter(P_dir.ravel(), P_loc.ravel(), s=12, alpha=0.6)
lims = [np.nanmin(P_dir), np.nanmax(P_dir)]
plt.plot(lims, lims, lw=1)  # y=x
plt.xlabel("Direct potential")
plt.ylabel("Multipole potential")
plt.title("Parity: P_mom vs P_dir")
plt.tight_layout()
plt.show()


# --- 1) Relative error vs p, one curve per separation ---
plt.figure(figsize=(7,5))
for j, R in enumerate(sep_dist):
    plt.plot(list(all_p), rel_err[:, j], marker='o', label=f'R={R}')
plt.yscale('log')
plt.xlabel('Expansion order p')
plt.ylabel('Relative error')
plt.title('Relative error vs p (per separation)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.tight_layout()

# --- 2) Relative error vs separation, one curve per p ---
plt.figure(figsize=(7,5))
for i, p in enumerate(all_p):
    plt.plot(sep_dist, rel_err[i, :], marker='o', label=f'p={p}')
plt.xscale('log', base=2)     # you used R = 2**i; base-2 makes spacing intuitive
plt.yscale('log')
plt.xlabel('Separation R (|center|)')
plt.ylabel('Relative error')
plt.title('Relative error vs separation (per p)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(ncol=2)
plt.tight_layout()

# --- 3) Heatmap of log10 relative error over (p, R) ---
plt.figure(figsize=(6.5,5))
im = plt.imshow(
    np.log10(rel_err),          # rows = p, cols = R
    origin='lower',
    aspect='auto',
    extent=[np.log2(sep_dist[0]), np.log2(sep_dist[-1]), all_p[0], all_p[-1]]
)
cbar = plt.colorbar(im, label=r'$\log_{10}(\mathrm{relative\ error})$')
plt.xlabel(r'$\log_2 R$')
plt.ylabel('Expansion order p')
plt.title('Error heatmap over (p, R)')
plt.tight_layout()

# --- Optional: magnitude/phase sanity plots (if complex) ---
if np.iscomplexobj(P_dir):
    phase_diff = np.angle(P_mom) - np.angle(P_dir)
    phase_diff = (phase_diff + np.pi) % (2*np.pi) - np.pi  # wrap to [-pi, pi]
    plt.figure(figsize=(7,4))
    for j, R in enumerate(sep_dist):
        plt.plot(list(all_p), np.abs(phase_diff[:, j]), marker='o', label=f'R={R}')
    plt.yscale('log')
    plt.xlabel('p'); plt.ylabel('abs phase diff [rad]')
    plt.title('Phase error vs p')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.tight_layout()

plt.show()
